This notebook shows the sequence of commands for an "end-to-end" verification. The idea is to show that we are able to 

    1 - Startup the telescope.
    2 - Acquire calibration data.
    3 - Initialize the observatory for the night.
    4 - Slew to a target.
    5 - Take an image of the target. 
    6 - Read the image from the butler and display it.
    7 - Shutdown the telescope.

The first step in this notebook is to load all the required libraries. 

In [1]:
import sys
import asyncio
import logging

import numpy as np

from lsst.ts import salobj


from lsst.ts.standardscripts.auxtel.attcs import ATTCS

In [69]:
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5

Before proceeding, add a logger and set logging level to `DEBUG` so we can get status message from the various tasks.

In [2]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

Next we initialize two high level classes to command the ATTCS and LATISS.

In [3]:
attcs = ATTCS()

Using selector: EpollSelector


In [4]:
await attcs.start_task

Read historical data in 0.01 sec
Read historical data in 0.47 sec
Read historical data in 0.70 sec
Read historical data in 1.03 sec
Read historical data in 1.76 sec
Read historical data in 2.41 sec
Read historical data in 4.59 sec
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 17 messages


[None, None, None, None, None, None, None]

In [ ]:
await asyncio.sleep(5)

In [ ]:
await salobj.set_summary_state(attcs.atmcs, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(attcs.atptg, salobj.State.ENABLED)

In [ ]:
await attcs.atptg.cmd_pointLoadModel.set_start(pointingFile="/home/saluser/auxtel_20190912.mod")

Since there is no reason do put these components in standby at a given time, let us assume `LATISS` is already enabled. The same is not true for ATTCS which must be put in standby at the end of the night so the telescope can be left in a safe mode. 

For that we call the `enable` method in ATTCS. This will leave the telescope and its subsystems ready to go. 

Note that the ATHexapod simulator is not working right now. For that we can tell attcs to skip it in enable and startup operations.

In [ ]:
ra = Angle("01:37:42.84548", unit=u.hour)
dec = Angle("-57:14:12.3101", unit=u.deg)
target_name="Alf Eri"
radec = ICRS(ra, dec)

In [ ]:
await attcs.enable(settings={
    'atmcs': "",
    'atptg': "",
    'ataos': "",
    'atpneumatics': "",
    'athexapod': "Default1",
    'atdome': "",
    'atdometrajectory': ""})

Select instrument port. This is not done at the startup.

In [ ]:
await attcs.atmcs.cmd_setInstrumentPort.set_start(port=1)

A trick to aways get a good target to point.

In [ ]:
time = await attcs.atptg.tel_timeAndDate.aget()

In [ ]:
time.lst, time.utc

In [ ]:
await attcs.atptg.cmd_stopTracking.start()

In [ ]:
ra = Angle("05 14 32.27210", unit=u.hour)
dec = Angle("-08 12 05.8981", unit=u.deg)
target_name="Bet Ori"
radec = ICRS(ra, dec)

In [ ]:
await attcs.slew_icrs(ra="05 14 32.27210", dec="-08 12 05.8981", target_name="Bet Ori", slew_timeout=120)

In [ ]:
attcs.slew_icrs?

In [ ]:
await attcs.slew_icrs(ra="05 17 36.3885646", dec="-06 50 39.870187", target_name="Tau Ori", slew_timeout=120)

In [82]:
attcs.check.atdome = False

In [96]:
await attcs.slew_icrs(ra="06 00 01.1299670717", dec="-08 06 27.161817102", rot_pa=-180., target_name="HD40672", slew_timeout=120)
 

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 13 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 62 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 12 messages
Flushing events
Scheduling check coroutines
Sending command
RemoteEvent(ATDome, 0, mainDoorState) falling behind; read 43 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 13 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 65 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 66 messages
RemoteTelemetry(ATMCS, 0, mount_AzEl_Encoders) falling behind; read 13 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 13 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 13 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 14 messages
[Telescope] delta Alt = -000.006 | del

In [70]:
ra = Angle("06 00 01.1299670717", unit=u.hour)
dec = Angle("-08 06 27.161817102", unit=u.deg)
target_name="HD40672"
radec = ICRS(ra, dec)


In [ ]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

In [71]:
# await attcs.atptg.cmd_offsetAzEl.set_start(el=-0.1*60.*60., 
#                                      az=0.456*60.*60., 
#                                      num=0)

RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 100 messages


In [ ]:
# await attcs.atptg.cmd_offsetRADec.set_start(off1=-0.348*60.*60.,
#                                             off2=0.317*60.*60., 
#                                      num=0)

In [132]:
await attcs.atptg.cmd_offsetAzEl.set_start(el=-0.364*60.*60., 
                                     az=0.292*60.*60., 
                                     num=0)

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, skyEnvironment) falling behind; read 82 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 100 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, azEl_mountMotorEncoders) falling behind; read 100 messages


In [ ]:
end_readout = await latiss.take_image(exptime=10., 
                                      shutter=True, 
                                      image_type="OBJECT", 
                                      group_id="TEST", 
                                      filter="disperser", 
                                      grating="ronchi170lpmm")

In [ ]:
print(end_readout)

Get visit id from the `end_readout` event. 

In [ ]:
visit = int(int(f"{end_readout.imageDate}{end_readout.imageNumber:05}"))
print(visit)

In [ ]:
await asyncio.sleep(15.)

In [ ]:
repo = os.path.join("/mnt/dmcs/oods_butler_repo/repo")
butler = dafPersist.Butler(repo)

In [ ]:
raw = butler.get("raw", visit=visit)

In [ ]:
display(raw.image, cmap=matplotlib.cm.gray, vmin=1e4, vmax=3e4)

Finaly, shutdown the system. 

In [ ]:
await attcs.shutdown()